In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-17"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8626,2024-10-17,Brasil Nbb,19:30,Bauru,Minas,2.37,1.49,156.5,1.80,1.86,3.5,1.93,1.74,S4N3zSjC,0.421941,0.671141,0.555556,0.537634,0.093082,1.0,0.4,NaN,NaN,136.322,22.206979,0.162901,132.048,45.899877,0.347600,128.264,137.030,135.78,205.20,0.0,0.0,0.0,0.0,0.322411,0.023184,0.073215,0.76,0.152,9.013158,0.745308,0.9,0.154692,-1.31,-0.262,-1.870229,0.743762,0.8,0.056238
8627,2024-10-17,Brasil Nbb,19:30,Pato,São Paulo,2.05,1.65,154.5,1.83,1.83,1.5,1.95,1.72,ELLBY5LO,0.487805,0.606061,0.546448,0.546448,0.093865,0.6,0.6,NaN,NaN,234.500,172.999068,0.737736,183.584,99.612803,0.542601,219.538,201.908,149.48,122.22,0.0,0.0,0.0,0.0,0.152888,0.000000,0.088629,2.03,0.406,2.586207,0.490020,0.5,0.009980,2.55,0.510,1.274510,0.558444,0.6,0.041556
8628,2024-10-17,Brasil Nbb,19:30,São José,Flamengo,3.38,1.25,151.5,1.88,1.78,7.5,1.95,1.72,6kZfxlMa,0.295858,0.800000,0.531915,0.561798,0.095858,0.4,0.2,NaN,NaN,114.660,21.163171,0.184573,109.714,29.095298,0.265192,114.660,109.714,84.70,79.06,0.0,0.0,1.0,1.0,0.650599,0.038640,0.088629,-0.77,-0.154,-15.454545,0.694982,0.5,-0.194982,0.45,0.090,2.777778,0.787350,0.7,-0.087350
8629,2024-10-17,Brasil Nbb,20:00,Franca,Brasília,1.08,5.91,160.5,1.83,1.83,-14.5,1.95,1.72,hniMh96t,0.925926,0.169205,0.546448,0.546448,0.095131,0.0,0.4,NaN,NaN,138.308,19.455870,0.140671,322.892,182.976343,0.566680,117.492,293.796,117.26,222.04,0.0,0.0,0.0,0.0,0.977203,0.000000,0.088629,0.55,0.110,0.727273,0.829288,0.9,0.070712,0.56,0.112,43.839286,0.230931,0.1,-0.130931
8630,2024-10-17,Brasil Nbb,20:00,Mogi,R10 Score Vasco da Gama,2.59,1.42,145.5,1.83,1.83,4.5,1.95,1.72,QakUjmyg,0.386100,0.704225,0.546448,0.546448,0.090326,0.4,0.2,NaN,NaN,144.742,45.742752,0.316030,155.692,31.385405,0.201587,144.742,149.900,99.40,121.60,0.0,0.0,0.0,0.0,0.412626,0.000000,0.088629,1.72,0.344,4.622093,0.438525,0.3,-0.138525,1.05,0.210,2.000000,0.448665,0.6,0.151335
8631,2024-10-17,Eua Nba,20:00,Indiana Pacers,Charlotte Hornets,1.41,2.75,229.5,1.91,1.91,-5.5,1.95,1.87,tGLbQntt,0.709220,0.363636,0.523560,0.523560,0.072856,0.6,0.6,NaN,NaN,232.294,79.528668,0.342362,224.078,82.170765,0.366706,198.522,265.572,155.44,345.45,0.0,0.0,0.0,0.0,0.455540,0.000000,0.029617,-2.19,-0.438,-0.936073,0.653302,0.7,0.046698,0.77,0.154,11.363636,0.349824,0.4,0.050176
8632,2024-10-17,Eua Nba,20:30,Dallas Mavericks,Milwaukee Bucks,1.73,2.00,217.5,1.91,1.91,-1.5,1.91,1.91,pE1OKrAH,0.578035,0.500000,0.523560,0.523560,0.078035,0.2,1.0,NaN,NaN,193.398,44.273591,0.228925,216.090,87.843289,0.406513,187.920,318.182,164.22,203.40,0.0,0.0,0.0,0.0,0.102369,0.000000,0.000000,-3.07,-0.614,-1.188925,0.594427,0.5,-0.094427,-5.00,-1.000,-1.000000,0.445419,0.1,-0.345419
8633,2024-10-17,Europa Euroliga,14:30,Alba Berlin,Fenerbahce,3.63,1.31,161.5,1.91,1.91,6.5,2.00,1.71,Ywhyj94L,0.275482,0.763359,0.523560,0.523560,0.038841,0.8,0.6,NaN,NaN,404.746,224.962361,0.555811,144.148,46.497208,0.322566,291.762,174.498,160.44,170.15,0.0,0.0,0.0,0.0,0.664165,0.000000,0.110545,-3.09,-0.618,-4.255663,0.343561,0.2,-0.143561,-0.73,-0.146,-2.123288,0.463839,0.3,-0.163839
8634,2024-10-17,Europa Euroliga,15:05,Maccabi Tel Aviv,Partizan,2.42,1.61,169.5,1.83,2.00,2.5,2.00,1.71,IBT7pmSr,0.413223,0.621118,0.546448,0.500000,0.034341,0.6,0.8,NaN,NaN,174.406,81.034469,0.464631,145.072,60.641571,0.418010,165.232,191.736,229.71,247.68,0.0,0.0,0.0,0.0,0.284246,0.062772,0.110545,2.16,0.432,3.287037,0.668197,0.9,0.231803,-5.00,-1.000,-0.610000,0.404

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,20:00,Eua Nba,Indiana Pacers,Charlotte Hornets,229.5,1.91,0.7007,Over
1,20:30,Eua Nba,Dallas Mavericks,Milwaukee Bucks,217.5,1.91,0.8336,Over
